# Overview Materi

Source: https://www.youtube.com/watch?v=LDRbO9a6XPU

Jelaskan secara singkat apa itu decision tree menurut pemahamanmu!

Decision Tree adalah salah satu algoritma machine learning berbentuk struktur pohon, yang digunakan untuk klasifikasi maupun regresi. Cara kerjanya seperti data dipilah secara bertahap dengan aturan pada setiap node. Node internal ialah kondisi atau pertanyaan. Cabang adalah hasil dari kondisi tersebut (Ya/Tidak). Daun (leaf) artinya hasil akhir berupa klasifikasi atau nilai. Jadi, decision tree membagi data menjadi kelompok-kelompok kecil berdasarkan fitur sehingga mempermudah pengambilan keputusan.

# Import Data & Libraries

In [3]:
from __future__ import print_function

# label kolom
header = ["color", "diameter", "label"]

# data training
training_data = [
    ['Green', 3, 'Apple'],
    ['Yellow', 3, 'Apple'],
    ['Red', 1, 'Grape'],
    ['Red', 1, 'Grape'],
    ['Yellow', 3, 'Lemon'],]

# data testing
testing_data = [
    ['Green', 3, 'Apple'],
    ['Yellow', 4, 'Apple'],
    ['Red', 2, 'Grape'],
    ['Red', 1, 'Grape'],
    ['Yellow', 3, 'Lemon'],]

# Fungsi Dasar

In [4]:
# fungsi mencari apa saja unique value dari suatu kolom
def unique_vals(rows, col_index):
    return set([row[col_index] for row in rows])

# contoh penggunaan
print("Unique values di kolom color:", unique_vals(training_data, 0))
print("Unique values di kolom diameter:", unique_vals(training_data, 1))
print("Unique values di kolom label:", unique_vals(training_data, 2))

Unique values di kolom color: {'Yellow', 'Red', 'Green'}
Unique values di kolom diameter: {1, 3}
Unique values di kolom label: {'Apple', 'Grape', 'Lemon'}


In [5]:
# fungsi Menghitung jumlah unique value dari suatu kolom
def class_counts(rows):
    counts = {}
    for row in rows:
        label = row[-1]
        if label not in counts:
            counts[label] = 0
        counts[label] += 1
    return counts

# contoh penggunaan
training_data = [
    ['Green', 3, 'Apple'],
    ['Yellow', 3, 'Apple'],
    ['Red', 1, 'Grape'],
    ['Red', 1, 'Grape'],
    ['Yellow', 3, 'Lemon'],
]
print(class_counts(training_data))

{'Apple': 2, 'Grape': 2, 'Lemon': 1}


In [6]:
# fungsi pengecekan suatu value numerik atau bukan
def is_numeric(value):
    return isinstance(value, int) or isinstance(value, float)

# contoh penggunaan
print(is_numeric(7))
print(is_numeric(3.5))
print(is_numeric("Apple"))
print(is_numeric("7"))

True
True
False
False


In [8]:
# kelas untuk merepresentasikan pertanyaan pada decision tree
class Question:

    # inisialisasi kolom dan nilai pertanyaan
    def __init__(self, column, value):
        self.column = column
        self.value = value

    # mengecek apakah contoh data sesuai dengan pertanyaan
    def match(self, example):
        val = example[self.column]
        if is_numeric(val):
            return val >= self.value
        else:
            return val == self.value

    # menampilkan pertanyaan dalam format string yang mudah dibaca
    def __repr__(self):
        condition = ">=" if is_numeric(self.value) else "=="
        return f"Apakah {header[self.column]} {condition} {self.value}?"

# contoh penggunaan 1
q1 = Question(0, "Green")
q2 = Question(1, 3)
print(q1)
print(q2)

# cek match dengan data
example = training_data[0]
print(q1.match(example))
print(q2.match(example))

Apakah color == Green?
Apakah diameter >= 3?
True
True


In [9]:
# membagi dataset menjadi dua berdasarkan pertanyaan
def partition(rows, question):
    true_rows, false_rows = [], []
    for row in rows:
        if question.match(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
    return true_rows, false_rows

# contoh penggunaan
q = Question(0, "Green")
true_rows, false_rows = partition(training_data, q)
print("True rows:", true_rows)
print("False rows:", false_rows)

True rows: [['Green', 3, 'Apple']]
False rows: [['Yellow', 3, 'Apple'], ['Red', 1, 'Grape'], ['Red', 1, 'Grape'], ['Yellow', 3, 'Lemon']]


**apa itu gini impurity?**
<br> gini impurity berfungsi mengukur tingkat ketidakmurnian atau ketidakteraturan pada sebuah simpul (node) dalam pohon

In [10]:
# menghitung nilai Gini Impurity untuk sebuah dataset
def gini(rows):
    counts = class_counts(rows)  # hitung jumlah tiap label
    impurity = 1
    for lbl in counts:
        prob_of_lbl = counts[lbl] / float(len(rows))
        impurity -= prob_of_lbl**2
    return impurity

# contoh penggunaan
print("Gini training_data:", gini(training_data))

# coba dataset kecil
example_rows = [
    ['Red', 1, 'Grape'],
    ['Red', 1, 'Grape'],
    ['Yellow', 3, 'Lemon']
]
print("Gini example_rows:", gini(example_rows))

Gini training_data: 0.6399999999999999
Gini example_rows: 0.4444444444444445


**apa itu information gain?**
<br> information gain berfungsi mengukur seberapa efektif sebuah fitur dalam memisahkan data berdasarkan kelas-kelasnya

In [11]:
# menghitung nilai Information Gain dari pemisahan dataset
def info_gain(left, right, current_uncertainty):
    p = float(len(left)) / (len(left) + len(right))
    return current_uncertainty - p * gini(left) - (1 - p) * gini(right)

# contoh penggunaan
# ambil pertanyaan untuk split
q = Question(0, 'Red')
true_rows, false_rows = partition(training_data, q)

# hitung info gain
current_uncertainty = gini(training_data)
gain = info_gain(true_rows, false_rows, current_uncertainty)

print("Information Gain untuk pertanyaan:", q, "adalah", gain)

Information Gain untuk pertanyaan: Apakah color == Red? adalah 0.37333333333333324


In [12]:
# mencari pertanyaan terbaik untuk membagi dataset berdasarkan information gain tertinggi
def find_best_split(rows):
    best_gain = 0
    best_question = None
    current_uncertainty = gini(rows)
    n_features = len(rows[0]) - 1

    for col in range(n_features):
        values = set([row[col] for row in rows])

        for val in values:
            question = Question(col, val)

            # splitting the dataset
            true_rows, false_rows = partition(rows, question)

            # Skip this split if it doesn't divide the dataset
            if len(true_rows) == 0 or len(false_rows) == 0:
                continue
            gain = info_gain(true_rows, false_rows, current_uncertainty)
            if gain >= best_gain:
                best_gain, best_question = gain, question
    return best_gain, best_question

# contoh penggunaan
best_gain, best_question = find_best_split(training_data)
print("Best Gain:", best_gain)
print("Best Question:", best_question)

Best Gain: 0.37333333333333324
Best Question: Apakah diameter >= 3?


# Fungsi Decision Tree

In [13]:
# merepresentasikan node daun (leaf) pada decision tree yang berisi hasil prediksi
class Leaf:

    # inisialisasi leaf dengan menghitung jumlah kemunculan tiap kelas
    def __init__(self, rows):
        self.predictions = class_counts(rows)

In [14]:
# merepresentasikan node keputusan (decision node) yang berisi pertanyaan dan cabang
class Decision_Node:

    # inisialisasi node dengan pertanyaan, cabang benar, dan cabang salah
    def __init__(self, question, true_branch, false_branch):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch

In [15]:
# membangun decision tree secara rekursif
def build_tree(rows):

    gain, question = find_best_split(rows)
    if gain == 0:
        return Leaf(rows)
    true_rows, false_rows = partition(rows, question)
    true_branch = build_tree(true_rows)
    false_branch = build_tree(false_rows)
    return Decision_Node(question, true_branch, false_branch)

In [16]:
# mencetak struktur decision tree secara rekursif dalam format teks
def print_tree(node, spacing=""):

    # base case: jika sudah mencapai leaf
    if isinstance(node, Leaf):
        print(spacing + "Predict", node.predictions)
        return

    # mencetak pertanyaan pada node saat ini
    print(spacing + str(node.question))

    # mencetak cabang true secara rekursif
    print(spacing + '--> True:')
    print_tree(node.true_branch, spacing + "  ")

    # mencetak cabang false secara rekursif
    print(spacing + '--> False:')
    print_tree(node.false_branch, spacing + "  ")

# contoh penggunaan
my_tree = build_tree(training_data)
print_tree(my_tree)

Apakah diameter >= 3?
--> True:
  Apakah color == Yellow?
  --> True:
    Predict {'Apple': 1, 'Lemon': 1}
  --> False:
    Predict {'Apple': 1}
--> False:
  Predict {'Grape': 2}


In [17]:
# mengklasifikasikan satu baris data menggunakan decision tree
def classify(row, node):

    # base case: jika sudah mencapai leaf
    if isinstance(node, Leaf):
        return node.predictions

    # menentukan apakah mengikuti cabang true atau cabang false
    # dengan membandingkan nilai fitur pada baris dengan pertanyaan di node
    if node.question.match(row):
        return classify(row, node.true_branch)
    else:
        return classify(row, node.false_branch)

# contoh penggunaan
my_tree = build_tree(training_data)
print(classify(testing_data[0], my_tree))  # contoh output: {'Apple': 1}
print(classify(testing_data[1], my_tree))

{'Apple': 1}
{'Apple': 1, 'Lemon': 1}


In [18]:
# menampilkan prediksi pada leaf dalam format persentase
def print_leaf(counts):
    total = sum(counts.values())
    probs = {}
    for lbl in counts:
        probs[lbl] = str(int(counts[lbl] / total * 100)) + "%"
    return probs

# contoh penggunaan
my_tree = build_tree(training_data)

prediction = classify(testing_data[0], my_tree)
print(print_leaf(prediction))

prediction = classify(testing_data[1], my_tree)
print(print_leaf(prediction))

{'Apple': '100%'}
{'Apple': '50%', 'Lemon': '50%'}


# Predict Using Decision Tree

In [19]:
# menguji decision tree dengan data uji dan membandingkan hasil prediksi dengan label asli
for row in testing_data:
    prediction = classify(row, my_tree)
    print("Data:", row)
    print("Prediksi:", print_leaf(prediction))
    print("Label Asli:", row[-1])
    print()

Data: ['Green', 3, 'Apple']
Prediksi: {'Apple': '100%'}
Label Asli: Apple

Data: ['Yellow', 4, 'Apple']
Prediksi: {'Apple': '50%', 'Lemon': '50%'}
Label Asli: Apple

Data: ['Red', 2, 'Grape']
Prediksi: {'Grape': '100%'}
Label Asli: Grape

Data: ['Red', 1, 'Grape']
Prediksi: {'Grape': '100%'}
Label Asli: Grape

Data: ['Yellow', 3, 'Lemon']
Prediksi: {'Apple': '50%', 'Lemon': '50%'}
Label Asli: Lemon

